<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/UH-Insure-NSA.git"
REPO="UH-Insure-NSA"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

In [ ]:
from model.test import test
test()

In [ ]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
os.chdir("/content")
!pip install -q transformers datasets

import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer

In [18]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen3-Coder-30B-A3B-Instruct"  # Model checkpoint on the Hugging Face Hub
DATASET = "/content/unsupervised_hybrid_data.jsonl" # Josh's preproccesed dataset.
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
MAX_STEPS = 200 #2000  # max_steps
BATCH_SIZE = 1  # batch_size
GR_ACC_STEPS = 8  # gradient_accumulation_steps
LR = 2e-4                     # learning_rate
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
NUM_WARMUP_STEPS = 15  # num_warmup_steps
EVAL_FREQ = 25
SAVE_FREQ = 50
LOG_FREQ = 10
OUTPUT_DIR = "peft-FT-3-Coder-30b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.25  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate

# LORA
LORA_R = 8  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [7]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [8]:
import torch
from tqdm import tqdm

SRC = "/content/unsupervised_hybrid_data.jsonl"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def detect_type(filename):
  if filename.endswith(".cry"):
    return "cryptol"
  elif filename.endswith(".saw"):
    return "saw"
  else:
    return "text"

dataset = load_dataset("json", data_files={"data": SRC})["data"]
dataset = dataset.map(lambda x: {"type": detect_type(x["filename"])})

cryptol = dataset.filter(lambda x: x["type"] == "cryptol")
saw = dataset.filter(lambda x: x["type"] == "saw")
text = dataset.filter(lambda x: x["type"] == "text")

cryptol_split = cryptol.train_test_split(test_size=0.1, seed=42)
saw_split = saw.train_test_split(test_size=0.1, seed=42)
text_split = text.train_test_split(test_size=0.1, seed=42)

train_ds = concatenate_datasets([cryptol_split["train"], saw_split["train"], text_split["train"]])
eval_ds = concatenate_datasets([cryptol_split["test"], saw_split["test"], text_split["test"]])

print(train_ds[0].keys())
assert DATA_COLUMN in train_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(eval_ds[0].keys())
assert DATA_COLUMN in eval_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(len(train_ds), len(eval_ds))

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_ds, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

Filter:   0%|          | 0/512 [00:00<?, ? examples/s]

Filter:   0%|          | 0/512 [00:00<?, ? examples/s]

dict_keys(['filename', 'filetype', 'content', 'variant', 'input_ids', 'n_tokens', 'set', 'type'])
dict_keys(['filename', 'filetype', 'content', 'variant', 'input_ids', 'n_tokens', 'set', 'type'])
460 52


100%|██████████| 400/400 [00:01<00:00, 299.45it/s]

The character to token ratio of the dataset is: 2.39


In [9]:
from model.dataset.util import *

In [10]:
from model.dataset.constantLengthDataset import ConstantLengthDataset
import torch

def make_overlapping_chunks(texts, tokenizer, seq_length=2048, stride=1024):
  tokens = tokenizer(texts, add_special_tokens=False)["input_ids"]
  flat = [token for seq in tokens for token in seq]
  chunks = []

  for i in range(0, len(flat) - seq_length, stride):
    chunk = flat[i : i + seq_length]
    chunks.append(tokenizer.decode(chunk))

  return chunks

overlap_train = make_overlapping_chunks(
    [item[DATA_COLUMN] for item in train_ds],
    tokenizer,
    seq_length=SEQ_LENGTH,
    stride=SEQ_LENGTH // 2
)

overlap_eval = make_overlapping_chunks(
    [item[DATA_COLUMN] for item in eval_ds],
    tokenizer,
    seq_length=SEQ_LENGTH,
    stride=SEQ_LENGTH // 2
)

train_ds_overlap = Dataset.from_dict({DATA_COLUMN: overlap_train})
eval_ds_overlap = Dataset.from_dict({DATA_COLUMN: overlap_eval})


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_ds_overlap,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        eval_ds_overlap,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [11]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

base = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        dtype=torch.bfloat16,
        device_map="auto",
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
        offload_folder="/content/offload"
)

model = prepare_model_for_kbit_training(base)

config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00001-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

model-00014-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00015-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00016-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [12]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = ["q_proj","k_proj","v_proj","o_proj", "gate"]

ATTN: Qwen3MoeAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=2048, bias=False)
  (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
  (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
)
MLP: Qwen3MoeSparseMoeBlock(
  (gate): Linear4bit(in_features=2048, out_features=128, bias=False)
  (experts): ModuleList(
    (0-127): 128 x Qwen3MoeMLP(
      (gate_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (up_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (down_proj): Linear4bit(in_features=768, out_features=2048, bias=False)
      (act_fn): SiLUActivation()
    )
  )
)


In [13]:
# Set up lora
from peft import PeftModel

peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

# model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b") # If your just trying to test the model.
model = get_peft_model(base, peft_config) # If you want to continue training.

model.print_trainable_parameters()

trainable params: 7,520,256 || all params: 30,539,642,880 || trainable%: 0.0246


In [19]:
train_ds.start_iteration = 0

training_args = TrainingArguments(
    output_dir=f"tam1444AH/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    report_to=["tensorboard"],
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    resume_from_checkpoint=False,
)

print(f"Training samples: {len(train_ds)}")
print(f"Sample text:\n\n{train_ds[0][DATA_COLUMN][:400]}")

Training samples: 460
Sample text:

/**
 * @copyright Galois, Inc.
 * @author John Christensen <jchristensen@galois.com>
 *
 * Parameters are taken from Table 3.1 of the Simon documentation.
 *
 */

module Primitive::Symmetric::Cipher::Block::Simon::Instantiations::Simon64_96 =
  Primitive::Symmetric::Cipher::Block::Simon::Specification where
    type n = 32
    type m = 3
    type T = 42
    type j = 2



In [20]:
import math

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

print("Training...")
trainer.train(resume_from_checkpoint=False)

eval_results = trainer.evaluate()
eval_loss = eval_results["eval_loss"]
perplexity = math.exp(eval_loss)
print(f"Eval loss = {eval_loss:.2f}, Perplexity = {perplexity:.2f}")

/tmp/ipython-input-1014676409.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training...


Step,Training Loss,Validation Loss
25,0.900300,1.449605
50,0.904500,1.425013
75,0.989400,1.412897
100,0.757300,1.408495
125,0.894200,1.403536
150,0.874900,1.401150
175,0.841000,1.400515
200,0.910300,1.400166


Eval loss = 1.40, Perplexity = 4.06


In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install -q transformers peft
!pip install -q transformers peft bitsandbytes accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Base model load
base = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-Coder-30B-A3B-Instruct",
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA fine-tuned adapter on top of it
model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Coder-30B-A3B-Instruct", trust_remote_code=True)
model.eval()


In [ ]:
from transformers import pipeline
import gc, torch
gc.collect()
torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="balanced_low_0",
    dtype="bfloat16",
)

prompt = """
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input block and returns a 256-bit digest."
"""
result = pipe(prompt, max_new_tokens=300)
print(result[0]["generated_text"])

prompts = [
    'Translate this English spec into Cryptol:\n\n"A function xor_inverse that proves (a XOR b) XOR b == a"',
    'Translate this English spec into Cryptol:\n\n"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"',
    'Translate this English spec into Cryptol:\n\n"Create a property proving that addition is commutative for 8-bit words."'
]

for p in prompts:
    print("\n=== Prompt ===")
    print(p)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    print("\n--- Generated Cryptol ---")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
prompt = """
Translate this English spec into SAW script:

"Verify that the function xor_inverse in xor_inverse.c returns (a XOR b) XOR b == a
for all 8-bit values of a and b."
"""
result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])

prompt = """
Generate both Cryptol and SAW verification code for this spec:
"A function add8 that proves addition is commutative for 8-bit words."
"""
result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])

prompt = """
Write a SAW script verifying that the Cryptol function sha256_hash implements
the same behavior as the C function sha256_hash_ref defined in sha256.c
using crucible_llvm_verify.
"""

result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])



In [ ]:
!apt-get update -q
!apt-get install -y wget tar

!wget -q https://github.com/GaloisInc/saw-script/releases/download/v1.3/saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!tar -xzf saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!mv saw-1.3-ubuntu-22.04-X64-with-solvers saw

!wget -q https://github.com/GaloisInc/cryptol/releases/download/3.3.0/cryptol-3.3.0-ubuntu-20.04-X64-with-solvers.tar.gz
!tar -xzf cryptol-3.3.0-ubuntu-22.04-X64-with-solvers.tar.gz
!mv cryptol-3.3.0-ubuntu-22.04-X64-with-solvers cryptol

In [ ]:
import os
os.environ["PATH"] = "/content/saw/bin:/content/cryptol/bin:" + os.environ["PATH"]

In [ ]:
!saw --version
!cryptol --version

In [ ]:
!git clone https://github.com/GaloisInc/cryptol-specs.git

In [ ]:
%cd /content/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES/Verifications
os.environ["CRYPTOLPATH"] = "/content/cryptol-specs"

In [ ]:
!saw AES128.saw

In [ ]:
%cd /content
!git clone https://github.com/plobethus/crypto-c.git

In [ ]:
%cd crypto-c

In [ ]:
!chmod +x scripts/colab_setup.sh
!bash scripts/colab_setup.sh
%cd /content